In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import pandas as pd

from imblearn.over_sampling import *
from imblearn.under_sampling import RandomUnderSampler
from focal_loss import BinaryFocalLoss
import numpy as np
import seaborn as sns


In [1]:

# 데이터 입력
df = pd.read_csv('sample_data/sonar.csv',header=None)

# sns.pairplot(df, diag_kind="kde")
# plt.show()



NameError: ignored

In [2]:

# 데이터 분류
dataset = df.values
X = dataset[:,0:60].astype(float)
Y_obj = dataset[:,60]

X = pd.DataFrame(1000*X)
Y_obj = pd.DataFrame(Y_obj)
# Y_obj =[ R,R,R,R,R,... M,M,M,M,]

# 문자열을 숫자로 변환
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

## 원핫 인코딩
Y_one_hot=tf.keras.utils.to_categorical(Y)




NameError: ignored

In [3]:

# 전체 데이터에서 학습 데이터와 테스트 데이터로 구분
X_train1, X_test, Y_train1, Y_test = train_test_split(X, Y, test_size=0.3,  shuffle=True)  ## shuffle=True로 하면 데이터를 섞어서 나눔
## 학습 셋에서 학습과 검증 데이터로 구분
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train1, Y_train1, test_size=0.2, shuffle=True)  ## shuffle=True로 하면 데이터를 섞어서 나눔


NameError: ignored

In [4]:

print("train class bin : " , np.bincount(Y_train.flatten().astype('int8')))
print("val class bin : " , np.bincount(Y_valid.flatten().astype('int8')))
print("test class bin : " , np.bincount(Y_test.flatten().astype('int8')))


NameError: ignored

In [5]:

# 모델의 설정
activation=tf.keras.activations.sigmoid
out_activation=tf.keras.activations.sigmoid
input_Layer = tf.keras.layers.Input(shape=(60,))
x = tf.keras.layers.Dense(120, activation=activation)(input_Layer)
x = tf.keras.layers.Dense(60, activation=activation)(x)
Out_Layer= tf.keras.layers.Dense(1, activation=out_activation)(x)

model = tf.keras.models.Model(inputs=[input_Layer], outputs=[Out_Layer])
model.summary()


NameError: ignored

In [6]:


# 모델 컴파일
loss=tf.keras.losses.binary_crossentropy
optimizer = tf.keras.optimizers.SGD(learning_rate=0.007)
metrics=tf.keras.metrics.binary_accuracy

model.compile(loss=loss,
             optimizer=optimizer,
             metrics=[metrics])

## model fit은 histoy를 반환한다. 훈련중의 발생하는 모든 정보를 담고 있는 딕셔너리.
result=model.fit(X_train, Y_train, epochs=1000, batch_size=100, validation_data=(X_valid,Y_valid)) # validation_data=(X_valid,Y_valid)을 추가하여 학습시 검증을 해줌.
## histoy는 딕셔너리이므로 keys()를 통해 출력의 key(카테고리)를 확인하여 무엇을 받고 있는지 확인.
print(result.history.keys())


NameError: ignored

In [7]:

### result에서 loss와 val_loss의 key를 가지는 값들만 추출
loss = result.history['loss']
val_loss = result.history['val_loss']
### loss와 val_loss를 그래프화
epochs = range(1, len(loss) + 1)
plt.subplot(211)  ## 2x1 개의 그래프 중에 1번째
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

### result에서 binary_accuracy와 val_binary_accuracy key를 가지는 값들만 추출
acc = result.history['binary_accuracy']
val_acc = result.history['val_binary_accuracy']

### binary_accuracy와 val_binary_accuracy key를 그래프화
plt.subplot(212)  ## 2x1 개의 그래프 중에 2번째
plt.plot(epochs, acc, 'ro', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
## 그래프 띄우기
plt.show()


NameError: ignored

In [ ]:

# model.evalueate를 통해 테스트 데이터로 정확도 확인하기.
## model.evaluate(X_test, Y_test)의 리턴값은 [loss, binary_acuuracy ]  -> 위 model.compile에서 metrics=[ keras.metrics.binary_accuracy]옵션을 주어서 binary acuuracy 출력됨.
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))
